<a href="https://colab.research.google.com/github/Rhqo/LLM_Study/blob/main/rlf(using_tlf)_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### RLF(Reinforment learning from feedback) using TRL(Transformer reinforcement learning) library

In [1]:
!pip install datasets -q
!pip install tqdm -q
!pip install wandb -q
!pip install trl -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 828.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from datasets import load_dataset
from tqdm import tqdm
import wandb
from tqdm import tqdm
import numpy as np
import pandas as pd
from random import choices
import matplotlib.pyplot as plt

tqdm.pandas()

from datasets import load_dataset

from transformers import AutoTokenizer, pipeline, AutoModelForSequenceClassification

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead, \
                create_reference_model, AutoModelForSeq2SeqLMWithValueHead
import os
import torch

In [5]:
flan_t5_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(config.model_name)
flan_t5_model_ref = create_reference_model(flan_t5_model)
flan_t5_tokenizer = AutoTokenizer.from_pretrained(config.model_name)

tokenizer = AutoTokenizer.from_pretrained("textattack/roberta-base-CoLA")
model = AutoModelForSequenceClassification.from_pretrained("textattack/roberta-base-CoLA")
cola_pipeline = pipeline("text-classification", model=model, tokenizer=tokenizer)
sentiment_pipeline = pipeline('text-classification', 'cardiffnlp/twitter-roberta-base-sentiment')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [8]:
dataset = load_dataset("argilla/news-summary")

dataset = dataset.map(
    lambda x: {"input_ids": flan_t5_tokenizer.encode('summarize: ' + x["text"], return_tensors="pt")},
    batched=False
    )
dataset = dataset.remove_columns(['metadata', 'status', 'event_timestamp', 'metrics', 'prediction', 'prediction_agent', 'annotation'])
dataset.set_format("pytorch")

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/20417 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (631 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/20417 [00:00<?, ? examples/s]

In [11]:
def collator(data):
  return dict((key, [d[key] for d in data]) for key in data[0])

In [12]:
def get_cola_scores(texts):
  scores = []
  results = cola_pipeline(texts, function_to_apply='none', top_k=None)
  for result in results:
    for label in result:
      if label['label'] == 'LABEL_1':
        scores.append(label['score'])
  return scores

def get_sentiment_scores(texts):
  scores = []
  results = sentiment_pipeline(texts, function_to_apply='none', top_k=None)
  for result in results:
    for label in result:
      if label['label'] == 'LABEL_1':
        scores.append(label['score'])
  return scores

In [13]:
config = PPOConfig(
    model_name = "google/flan-t5-small",
    batch_size = 8,
    mini_batch_size = 1,
    learning_rate = 2e-5,
    remove_unused_columns = False, log_with = "wandb",
    gradient_accumulation_steps = 8
)

ppo_trainer = PPOTrainer(
    config, flan_t5_model, flan_t5_model_ref, flan_t5_tokenizer, dataset['train'], data_collator=collator
)

generation_kwargs = {
    "min_length": 64,
    "num_beams": 5,  # lookahead parameter
    "no_repeat_ngram_size": 5,  # prescence penalty
    "do_sample": True,
    "pad_token_id": flan_t5_tokenizer.pad_token_id,
    "max_length": 256,
    "eos_token_id": flan_t5_tokenizer.eos_token_id,
}

np.random.seed(42)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [14]:
for epoch in tqdm(range(2)):
  for batch in tqdm(ppo_trainer.dataloader):
    game_data = dict()
    game_data['query'] = ['summarize: ' + b for b in batch["text"]]

    input_tensors = [_.squeeze() for _ in batch["input_ids"]]
    response_tensors = []
    for query in input_tensors:
      response = ppo_trainer.generate(query.squeeze(), **generation_kwargs)
      response_tensors.append(response.squeeze())

    game_data["response"] = [flan_t5_tokenizer.decode(r.squeeze(), skip_special_tokens=False) for r in response_tensors]

    game_data["clean_respose"] = [flan_t5_tokenizer.decode(r.squeeze(), skip_special_tokens=True) for r in response_tensors]
    game_data['cola_scores'] = get_cola_scores(game_data["clean_respose"])
    game_data['neutral_scores'] = get_sentiment_scores(game_data["clean_respose"])

    rewards = game_data['neutral_scores']

    transposed_lists = zip(game_data['cola_scores'], game_data['neutral_scores'])

    rewards = [1*values[0] + 0.5*values[1] for values in transposed_lists]
    rewards = [torch.tensor([_]) for _ in rewards]

    stats = ppo_trainer.step(input_tensors, response_tensors, rewards)
    stats['env/reward'] = np.mean([r.cpu().numpy() for r in rewards])
    ppo_trainer.log_stats(stats, game_data, rewards)


  2%|▏         | 2/125 [01:14<1:14:16, 36.23s/it]/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1308: UserWarning: KL divergence is starting to become negative: -1.17 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(

  2%|▏         | 3/125 [01:36<59:51, 29.44s/it]  /usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1308: UserWarning: KL divergence is starting to become negative: -3.30 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(

  3%|▎         | 4/125 [02:03<57:48, 28.66s/it]/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1308: UserWarning:

In [15]:
flan_t5_model.save_pretrained("flan-t5-model")
flan_t5_tokenizer.save_pretrained("t5-align")

('t5-align/tokenizer_config.json',
 't5-align/special_tokens_map.json',
 't5-align/spiece.model',
 't5-align/added_tokens.json',
 't5-align/tokenizer.json')

In [16]:
nytimes = '''Merriam-Webster notes that a “Pandora’s box” can be “anything that looks ordinary but may produce unpredictable harmful results.” I’ve been thinking a lot about Pandora’s boxes lately, because we Homo sapiens are doing something we’ve never done before: lifting the lids on two giant Pandora’s boxes at the same time, without any idea of what could come flying out.

One of these Pandora’s boxes is labeled “artificial intelligence,” and it is exemplified by the likes of ChatGPT, Bard and AlphaFold, which testify to humanity’s ability for the first time to manufacture something in a godlike way that approaches general intelligence, far exceeding the brainpower with which we evolved naturally.

The other Pandora’s box is labeled “climate change,” and with it we humans are for the first time driving ourselves in a godlike way from one climate epoch into another. Up to now, that power was largely confined to natural forces involving Earth’s orbit around the sun.

For me the big question, as we lift the lids simultaneously, is: What kind of regulations and ethics must we put in place to manage what comes screaming out?

Let’s face it, we did not understand how much social networks would be used to undermine the twin pillars of any free society — truth and trust. So if we approach generative A.I. just as heedlessly — if we again go along with Mark Zuckerberg’s reckless mantra at the dawn of social networks, “move fast and break things” — oh, baby, we are going to break things faster, harder and deeper than anyone can imagine.

“There was a failure of imagination when social networks were unleashed and then a failure to responsibly respond to their unimagined consequences once they permeated the lives of billions of people,” Dov Seidman, the founder and chairman of the HOW Institute for Society and LRN, told me. “We lost a lot of time — and our way — in utopian thinking that only good things could come from social networks, from just connecting people and giving people a voice. We cannot afford similar failures with artificial intelligence.”'''.strip()

In [20]:
trained_tokenizer = AutoTokenizer.from_pretrained("t5-align")
new_summarizer = pipeline("summarization", 't5-align', tokenizer=trained_tokenizer)

OSError: t5-align does not appear to have a file named config.json. Checkout 'https://huggingface.co/t5-align/tree/None' for available files.

In [21]:
summary_list, sentiment_scores, cola_scores = [], [], []

for i in range(10):
    results = new_summarizer(f"{nytimes}", **generation_kwargs)
    summary_text = results[0]['summary_text']

    if summary_text.endswith('.') and '...' not in summary_text:
        print(f"Generated Summary {i+1}: {summary_text}")

        sentiment_score = get_neutral_scores([summary_text])
        sentiment_scores.extend(sentiment_score)
        print(f"Neutral Rewards for Summary {i+1}: {sentiment_score}")

        cola_score = get_cola_scores([summary_text])
        cola_scores.extend(cola_score)
        print(f"Cola Rewards for Summary {i+1}: {cola_score}")

        print('---------------------------')

average_sentiment_score = sum(sentiment_scores) / len(sentiment_scores)
average_cola_score = sum(cola_scores) / len(cola_scores)

print(f'Average Neutral Reward: {average_sentiment_score}')
print(f'Average Cola Reward: {average_cola_score}')
print('---------------------------')


NameError: name 'new_summarizer' is not defined